# The realization of TF-IDF for sentences

**document will be composed of api callings and registry operations**

In [2]:
from nltk.corpus import stopwords
import pickle

In [3]:
# combine the title with the values
def splice_key_value(file):
    registry_values = []
    with open(file,'rb') as fr:
        registry_values_dict = pickle.load(fr, encoding="bytes")
    for key, values in registry_values_dict.items():
        for value in values:
            registry_values.append(key+'_'+str(value))
    return registry_values

In [4]:
registry_values_ransom_file = '../dataset/registry_values/registry_values_ransom.pkl'
registry_values_normal_file = '../dataset/registry_values/registry_values_normal.pkl'

registry_values_ransom_list = splice_key_value(registry_values_ransom_file)
registry_values_normal_list = splice_key_value(registry_values_normal_file)

In [5]:
len(registry_values_ransom_list)

50515

In [6]:
len(set(registry_values_ransom_list))

6657

In [7]:
registry_values_ransom_list[:5]

['regkey_opened_HKEY_LOCAL_MACHINE\\System\\CurrentControlSet\\Services\\DnsCache\\Parameters',
 'regkey_opened_HKEY_LOCAL_MACHINE\\Software\\Policies\\Microsoft\\Windows NT\\DnsClient',
 'regkey_opened_HKEY_LOCAL_MACHINE\\Software\\Microsoft\\Tracing',
 'regkey_opened_HKEY_LOCAL_MACHINE\\Software\\Policies\\Microsoft\\System\\DNSClient',
 'regkey_opened_HKEY_LOCAL_MACHINE\\System\\CurrentControlSet\\Services\\Tcpip\\Parameters']

In [36]:
import numpy as np

# function to convert string to lower case and delete the stop words in document
def preprocess_string(document):
    stop_words = set(stopwords.words('english'))
    # convert to lowercase
    registry_values_list = np.char.lower(document)
    # split sentences into words
    registry_words = []
    for registry_value in registry_values_list:
        words = registry_value.split("\\")
        for word in words:
            registry_words.append(word)
    # delete stop words in them
    for stop_word in stop_words:
        try:
            registry_words.remove(stop_word)
        except:
            continue
        finally:
            return registry_words


In [37]:
# here, we count how many times a word appears in the words array
# we do not use n-grams,cuz there are single string not only word
from collections import Counter

def countWords(words):
    '''
        return: a dictionary of {WORD: COUNT} where count is how many times
        that word appears in "words"
    '''
    words_dict ={}
    word_dict = Counter(words)
    return word_dict


In [38]:
# calculate the frequency of term in a document
def termFrequency(document):
    word_freq = {}
    # get the words without stop words
    registry_words = preprocess_string(document)
    # get the count for terms
    word_dict = countWords(registry_words)
    for key, value in word_dict.items():
        word_freq[key] = word_dict[key]/len(registry_words)
    return word_freq

In [39]:
word_freq_ransom = termFrequency(registry_values_ransom_list)

In [47]:
# calculate the sentence importance
def senFrequency(sentence, word_freq):
    # split the sentence into words
    words_list = sentence.split('\\')
    words_list = np.char.lower(words_list)
    # log the frequency of sentences
    sen_freq_total = 0
    # log the number of non stop words in each process
    num = 0
    for word, freq in word_freq.items():
        if word in words_list:
            sen_freq_total += word_freq[word]
            num += 1
    sen_freq = np.round(sen_freq_total/num ,3)
    return sen_freq

In [ ]:
# # generate the sen_freq dict
# sen_freq_ran_dict = {}
# for i, sentence in enumerate(registry_values_ransom_list): 
#     # input the lowercased sentence
    
#     sen_freq = senFrequency(sentence, word_freq_ransom)
#     sen_freq_ran_dict[str(i)] = sen_freq

In [ ]:
# generate the sen_freq dict
sen_freq_ran_dict = {}
for sentence in registry_values_ransom_list: 
    # input the lowercase sentence
    sen_freq = senFrequency(sentence, word_freq_ransom)
    sen_freq_ran_dict[sentence] = sen_freq

In [ ]:
sen_freq_ran_dict

# Reference：
**Automated Behavioral Analysis of Malware A Case Study of WannaCry Ransomware**